In [11]:
import py3Dmol

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - gro_data: The entire contents of the .gro file as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    box_dims = (0.0, 0.0, 0.0)
    try:
        with open(filename, "r") as file:
            gro_data = file.read()
            
            # Extract the last line for box dimensions
            lines = gro_data.strip().split("\n")
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                # Convert box dimensions from nm to Å
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                print("Box dimensions line is invalid.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return gro_data, box_dims

def visualize_gro_with_box_units_excluding_sol(gro_data, box_dims):
    """
    Visualizes the .gro file with simulation box edges using py3Dmol.
    Excludes only SOL molecules from visualization.
    Args:
        - gro_data: The .gro file contents as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    if box_dims == (0.0, 0.0, 0.0):
        print("Invalid box dimensions. Cannot draw simulation box.")
        return

    viewer = py3Dmol.view(width=800, height=600)

    # Filter out only SOL molecules
    filtered_gro_data = "\n".join(
        line for line in gro_data.splitlines()
        if not line.startswith("SOL")
    )

    # Add the molecular structure
    viewer.addModel(filtered_gro_data, "gro")  # Load the filtered .gro file
    viewer.setStyle({"sphere": {"scale": 0.3}})  # Atoms as spheres

    # Add the box edges (in Å)
    x_len, y_len, z_len = box_dims
    box_lines = [
        [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
        [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],  # Bottom face
        [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
        [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],  # Top face
        [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
        [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]  # Vertical edges
    ]
    for line in box_lines:
        viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                            "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                            "radius": 0.1, "color": "gray"})

    # Center and zoom the viewer
    viewer.zoomTo()

    # Show the viewer
    return viewer.show()

def main():
    # Input .gro file
    gro_filename = "/mnt/new_volume/newsitt/topic1/dtab/nvt.gro"

    # Read the .gro file and box dimensions
    gro_data, box_dims = read_gro_file_with_box_and_units(gro_filename)

    # Visualize the .gro file with the simulation box, excluding only SOL
    visualize_gro_with_box_units_excluding_sol(gro_data, box_dims)

if __name__ == "__main__":
    main()


Extracted box dimensions (in Å): (36.7528, 36.7528, 36.7528)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
import py3Dmol

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - gro_data: The entire contents of the .gro file as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    box_dims = (0.0, 0.0, 0.0)
    try:
        with open(filename, "r") as file:
            gro_data = file.read()
            
            # Extract the last line for box dimensions
            lines = gro_data.strip().split("\n")
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                # Convert box dimensions from nm to Å
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                print("Box dimensions line is invalid.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return gro_data, box_dims

def visualize_gro_with_box_units(gro_data, box_dims):
    """
    Visualizes the .gro file with simulation box edges using py3Dmol.
    Converts the box dimensions and coordinates to angstroms.
    Args:
        - gro_data: The .gro file contents as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    if box_dims == (0.0, 0.0, 0.0):
        print("Invalid box dimensions. Cannot draw simulation box.")
        return

    viewer = py3Dmol.view(width=800, height=600)
    
    # Add the molecular structure from the .gro file
    viewer.addModel(gro_data, "gro")  # Load .gro file
    viewer.setStyle({"sphere": {"scale": 0.3}})  # Atoms as spheres
    
    # Add the box edges (in Å)
    x_len, y_len, z_len = box_dims
    box_lines = [
        [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
        [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],  # Bottom face
        [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
        [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],  # Top face
        [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
        [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]  # Vertical edges
    ]
    for line in box_lines:
        viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                            "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                            "radius": 0.1, "color": "gray"})

    # Center and zoom the viewer
    viewer.zoomTo()
    
    # Show the viewer
    return viewer.show()

def main():
    # Input .gro file
    gro_filename = "/mnt/new_volume/newsitt/topic1/dtab/nvt.gro"
    
    # Read the .gro file and box dimensions
    gro_data, box_dims = read_gro_file_with_box_and_units(gro_filename)
    
    # Visualize the .gro file with the simulation box
    visualize_gro_with_box_units(gro_data, box_dims)

if __name__ == "__main__":
    main()


Extracted box dimensions (in Å): (36.7528, 36.7528, 36.7528)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
from ase.io import read
from ase.visualize import view
from ase import Atoms

def load_gro_with_box(filename):
    """
    Reads a .gro file and extracts atomic positions along with box dimensions.
    Returns:
        - atoms: ASE Atoms object containing atomic positions and box dimensions.
    """
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
            
            # Parse the last line for box dimensions (in nm)
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                box_dims = [float(dim) * 10 for dim in box_line[:3]]  # Convert nm to Å
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                box_dims = None
                print("Box dimensions not found or invalid.")
            
            # Load the atomic structure using ASE
            atoms = read(filename)
            
            # Assign box dimensions to the ASE Atoms object
            if box_dims:
                atoms.set_cell(box_dims)  # Set periodic box
                atoms.set_pbc(True)       # Enable periodic boundary conditions
            
            return atoms
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None

def main():
    # Input .gro file
    gro_filename = "/mnt/new_volume/newsitt/topic1/dtab/box.gro"
    
    # Load the structure with box dimensions
    atoms = load_gro_with_box(gro_filename)
    
    # Visualize the structure
    if atoms:
        view(atoms)
    else:
        print("Failed to load the .gro file.")

if __name__ == "__main__":
    main()


Extracted box dimensions (in Å): [36.7528, 36.7528, 36.7528]
